# Wykrywanie wariantów DNA

na bazie 
https://app.terra.bio/#workspaces/help-gatk/GATKTutorials-Germline-June2019/notebooks/launch/1-gatk-germline-variant-discovery-tutorial.ipynb


In [1]:
#####
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://storage.googleapis.com/gatk-tutorials/workshop_1906/2-germline/images/vd-image1.png")

## Ustawienia zmiennych

In [18]:
import os
user_name = os.environ.get('USER')
print(user_name)
%env JAVA_HOME=/data/tools/jdk1.8.0_231/
os.environ['PATH'] =   '/data/tools/jdk1.8.0_231/bin/' + os.pathsep + os.environ['PATH']
! java -version

workspace = "/home/"+user_name+"/edugen_genomics"
os.environ['WORKSPACE'] = workspace
! echo ${WORKSPACE}

jupyter-tgambin
env: JAVA_HOME=/data/tools/jdk1.8.0_231/
java version "1.8.0_231"
Java(TM) SE Runtime Environment (build 1.8.0_231-b11)
Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
/home/jupyter-tgambin/edugen_genomics


In [ ]:
! rm sandbox/motherHC.vcf

## Wykrywanie wariantów GATK HaplotypeCaller w trybie standardowym
¶
Wywolanie GATK HC w najprostszej wersji dla pojedynczej próbki

In [3]:
! gatk HaplotypeCaller \
    -R ref/ref.fasta \
    -I bam/mother.bam \
    -O sandbox/motherHC.vcf \
    -L 20:10,000,000-10,200,000

Using GATK jar /opt/gatk-4.1.9.0/gatk-package-4.1.9.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /opt/gatk-4.1.9.0/gatk-package-4.1.9.0-local.jar HaplotypeCaller -R ref/ref.fasta -I bam/mother.bam -O sandbox/motherHC.vcf -L 20:10,000,000-10,200,000
11:50:52.463 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/opt/gatk-4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
11:50:53.206 INFO  HaplotypeCaller - ------------------------------------------------------------
11:50:53.207 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.1.9.0
11:50:53.207 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
11:50:53.207 INFO  HaplotypeCaller - Executing as jovyan@jupyter-matdaw on Linux v4.19.112+ amd64
11:50:53.207 INFO  HaplotypeCaller -

## Kompresja pliku oraz utworzenie indeksu

In [1]:
#! rm ${WORKSPACE}/sandbox/motherHC.vcf.gz
! bgzip sandbox/motherHC.vcf
! tabix sandbox/motherHC.vcf.gz
! ls sandbox/

fastqc_reports	motherHC.vcf.gz  motherHC.vcf.gz.tbi  motherHC.vcf.idx


## Wizualizacja BAM i VCF w programie IGV

In [18]:
from igv_jupyterlab import IGV

IGV(genome="hg19")

track = IGV.create_track(
    type = "alignment",
    name = "BAM1",
    url = "bam/mother.bam",
    #format = "bam",
    indexed = True
)
print(track)
IGV.load_track(track)

#b.load_track(
#    {
#        "type": "alignment",
#        "name": "mother BAM",
#        "url": "../edugen_genomics/bam/mother.bam",
#        "indexURL": "../edugen_genomics/bam/mother.bai",        
#        "format": "bam",
#        "indexed": True
#    })

#b.load_track(
#    {
#        "name": "mother HC VCF",
#        "url": "../edugen_genomics/sandbox/motherHC.vcf.gz",
#        "indexURL":"../edugen_genomics/sandbox/motherHC.vcf.gz.tbi",
#        "format": "vcf",
#        "type": "variant",
#        "indexed": True,
#        "visibilityWindow": 1000
#    })
#
#b.search('20:10002294-10002623')

{'name': 'BAM1', 'url': 'bam/mother.bam', 'indexURL': None, 'indexed': True, 'format': None, 'order': None, 'color': None, 'height': None, 'autoHeight': None, 'minHeight': None, 'maxHeight': None, 'type': 'alignment'}


TypeError: load_track() missing 1 required positional argument: 'track'

<div class="alert alert-block alert-warning">
<b>Zadanie 1:</b> Jaka jest zygotyczność dwóch widocznych wariantów? Czy jest ona zgodna z tym co widzimy w odczytach ? 
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 2:</b> Kliknij na ikonę edycji track'u z odczytami i zaznacz opcję "Soft clip reads". Algorytm mapowania (BWA MEM) zdecydował, że kara za obcięcie odczytów (Soft clipping) będzie mniejsza niż wstawienie insersji/delecji. 
</div>



## Ponowne uliniowienie odczytów i zasemblowane haplotypy

HaplotypeCaller posiada parametr -bamout pozwalający na wyświetlenie ponownie uliniowionych odczytów,
które następnie są wykorzystywane do wykrywania wariantów.


In [ ]:
! gatk HaplotypeCaller \
    -R ${WORKSPACE}/ref/ref.fasta \
    -I ${WORKSPACE}/bam/mother.bam\
    -O ${WORKSPACE}/sandbox/motherHCdebug.vcf \
    -bamout ${WORKSPACE}/sandbox/motherHCdebug.bam \
    -L 20:10,002,000-10,003,000

In [ ]:
import igv

b = igv.Browser({"genome": "hg19"})
b.show()

b.load_track(
    {
        "type": "alignment",
        "name": "original BAM",
        "url": "../edugen_genomics/bam/mother.bam",
        "indexURL": "../edugen_genomics/bam/mother.bai",        
        "format": "bam",
        "indexed": True
    })

b.load_track(
    {
        "type": "alignment",
        "name": "HC realigned BAM",
        "url": "../edugen_genomics/sandbox/motherHCdebug.bam",
        "indexURL": "../edugen_genomics/sandbox/motherHCdebug.bai",        
        "format": "bam",
        
        "indexed": True
    })

b.load_track(
    {
        "name": "mother HC VCF",
        "url": "../edugen_genomics/sandbox/motherHC.vcf.gz",
        "indexURL":"../edugen_genomics/sandbox/motherHC.vcf.gz.tbi",
        "format": "vcf",
        "type": "variant",
        "indexed": True,
        "visibilityWindow": 1000
    })

b.search('20:10002294-10002623')

Ponieważ jesteśmy zainteresowani jednym miejscem (20:10,002,000-10,003,000) to dokonalismy reuliniowienia tylko tego fragmentu. 

Po ponownym uliniowieniu zniknęły obcięte odczyty (soft clipped reads) . 
HaplotypeCaller wykorzystuje odczyty 'soft-clipped'  do ponownego uliniowienia. 
Z analizy odczytów wynika, że insercja i pobliski SNP są ze sobą sfazowane. Wynika z tego, 
że HaplotypeCaller znalazł różne możliwe uliniowienia po wykonaniu lokalnej reasemblacji. 
Po reasemblacji występuje wystarczająco dużo odczytów wskazujących na występowanie insercji, która byłaby pominięta w przypadku użycia prostszych algorytmów, nie wykorzystujących reasemblacji (np. UnifiedGenotyper)


<div class="alert alert-block alert-warning">
<b>Zadanie 3:</b> Przyjrzyj się insercji w poszczególnych odczytach.  Czy wszędzie jest ona taka sama?
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 4:</b> W jaki sposób wyświetlić w IGV haplotypy utworzone przez HC?
</div>

In [ ]:
import igv

b = igv.Browser({"genome": "hg19"})
b.show()

b.load_track(
    {
        "type": "alignment",
        "name": "original BAM",
        "url": "../edugen_genomics/bam/mother.bam",
        "indexURL": "../edugen_genomics/bam/mother.bai",        
        "format": "bam",
        "indexed": True
    })

b.load_track(
    {
        "type": "alignment",
        "name": "HC realigned BAM",
        "url": "../edugen_genomics/sandbox/motherHCdebug.bam",
        "indexURL": "../edugen_genomics/sandbox/motherHCdebug.bai",        
        "format": "bam",
         "colorBy": "tag",
        "colorByTag": "RG",
        "indexed": True
    })

b.load_track(
    {
        "name": "mother HC VCF",
        "url": "../edugen_genomics/sandbox/motherHC.vcf.gz",
        "indexURL":"../edugen_genomics/sandbox/motherHC.vcf.gz.tbi",
        "format": "vcf",
        "type": "variant",
        "indexed": True,
        "visibilityWindow": 1000
    })

b.search('20:10002294-10002623')

Czerwone odczyty nie są prawdziwe - (spójrz na RG). Są to odczyty reprezentujące haplotypy utworzone przez HaplotypeCaller. 
<div class="alert alert-block alert-warning">
<b>Zadanie 5:</b> Podejrzyj wsparcie dla każdego haplotypu poprzez pokolorowanie po tag'u HC zamiast RG. HaplotypeCaller oznacza odpowiednim tagiem HC wszystkie odczyty jednoznacznie wskazujący na dany "sztuczny haplotyp". Co teraz sądzisz o decyzji Variant Caller'a w sprawie wyboru typu insercji?
</div>

<div class="alert alert-block alert-warning">
<b>Zadanie 6:</b> Oddal widok i zobacz ile różnych haplotypów zostało znalezionych dla kolejnych trzech spójnych regionów. 
</div>

## Jednoczesne genotypowanie wielu próbek przy wykorzystaniu gVCF

Istnieje możliwość jednoczesnego genotypowania wielu probek w normalnym trybie pracy działania HaplotypeCallera (joint calling). Jednakże nie jest to rozwiązanie skalowalne. Dlatego rozdzielono przetwarzanie plików BAM od właściwego genotypowania poprzez wprowadzenie tzw. plików gVCF. 

W trybie potoku GVCF HaplotypeCaller urachmiany jest z opcją -ERC GVCF dla każdego pliku BAM, co w wyniku generuje pliki z rozszerzeniem .gvcf posiadające poza informacją o wariantach dodatkowe dane o jakości pozostałych regionów. Następnie jednoczesne genotypowanie wielu próbek odbywa się poprzez wykonanie metody GenotypeGVCF na wielu plikach gvcf. 

Utwórzmy plik g.vcf dla jednego pliku bam (mother.bam).

In [ ]:
! gatk HaplotypeCaller \
    -R ${WORKSPACE}/ref/ref.fasta \
    -I ${WORKSPACE}/bam/mother.bam \
    -O ${WORKSPACE}/sandbox/motherHC.g.vcf \
    -ERC GVCF \
    -L 20:10,000,000-10,200,000

In [ ]:
! bgzip ${WORKSPACE}/sandbox/motherHC.g.vcf
! tabix ${WORKSPACE}/sandbox/motherHC.g.vcf.gz

In [ ]:
import igv

b = igv.Browser({"genome": "hg19"})
b.show()


b.load_track(
    {
        "type": "variant",
        "name": "mother gvcf",
        "url": "../edugen_genomics/sandbox/motherHC.g.vcf.gz",
        "indexURL": "../edugen_genomics/sandbox/motherHC.g.vcf.gz.tbi",        
        "format": "vcf",        
        "indexed": True
    })

b.search('20:10002294-10002623')

Co sie zmienilo w stosunku do VCF?
Poza wariantami pojawily sie szare bloki reprezentujace stopien pewnosci, ze w danym regionie nie ma wariantu. 
Zmiana bloku oznacza zmiane pewnosci, ze w danym regionie nie ma wariantow. 


In [ ]:
# Podejrzyj kilka wierszy g.vcf
! zcat ${WORKSPACE}/sandbox/motherHC.g.vcf.gz |  tail

Jezeli podejrzymy zawartosc gVCF, to zobaczymy ze w kolumnie ALT pojawia sie pozycja <NON_REF>.
Identyfikator PL w kolumnie format oznacza prawdopodobienstwo wystepowania kolejnych alleli, wlaczajac <NON_REF>. 


In [ ]:
! gatk GenomicsDBImport \
    -V ${WORKSPACE}/sandbox/motherHC.g.vcf.gz \
    -V ${WORKSPACE}/gvcf/father.g.vcf.gz \
    -V ${WORKSPACE}/gvcf/son.g.vcf.gz \
    --genomicsdb-workspace-path ${WORKSPACE}/sandbox/trio \
    --intervals 20:10,000,000-10,200,000

In [ ]:
! rm -r sandbox
! ln -s ${WORKSPACE}/sandbox/ sandbox

In [ ]:
# z bazy mozemy wyciagnac polaczony gvcf
! gatk SelectVariants \
    -R ${WORKSPACE}/ref/ref.fasta \
    -V gendb://sandbox/trio \
    -O ${WORKSPACE}/sandbox/trio_selectvariants.g.vcf

In [ ]:
! cat ${WORKSPACE}/sandbox/trio_selectvariants.g.vcf

Ostatnim krokiem jest genotyopwanie

In [ ]:
! gatk GenotypeGVCFs \
    -R /home/${USER}/2-germline-vd/ref/ref.fasta \
    -V gendb://sandbox/trio \
    -O ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -L 20:10,000,000-10,200,000

In [ ]:
! bgzip ${WORKSPACE}/sandbox/trioGGVCF.vcf 
! tabix ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz

In [ ]:
import igv

b = igv.Browser({"genome": "hg19"})
b.show()



b.load_track(
    {
        "type": "variant",
        "name": "combined gvcf",
        "url": "../edugen_genomics/sandbox/trioGGVCF.vcf.gz",
        "indexURL":  "../edugen_genomics/sandbox/trioGGVCF.vcf.gz.tbi",  
        "format": "vcf",        
        "indexed": True
    })

b.search('20:10002294-10002623')

In [ ]:
! cat ${WORKSPACE}/sandbox/trioGGVCF_metrics

# Analiza pokrycia

In [ ]:
import os
import pysam
with pysam.AlignmentFile(workspace+'/bam/mother.bam', 'rb') as samfile:
    for pileupcolumn in samfile.pileup("20", 10002294, 10002623):
        #if (pileupcolumn.pos != 16099988):   continue;
        print ("\ncoverage at base %s = %s" % (pileupcolumn.pos, pileupcolumn.n))

In [ ]:
 ! mosdepth    ${WORKSPACE}/sandbox/mother_coverage ${WORKSPACE}/bam/mother.bam

In [ ]:
! ls  ${WORKSPACE}/sandbox/

In [ ]:
import pandas as pd

tsv_read = pd.read_csv(workspace+'/sandbox/mother_coverage.per-base.bed.gz', sep='\t' , header=None)

In [ ]:
tsv_read.columns = ["chr", "start", "end", "coverage"]
tsv_read

<div class="alert alert-block alert-warning">
<b>Zadanie 7:</b> Zrób wykres głębokości pokrycia dla pozycji (chr20:10002294-10002623)i porownaj go z wizualizacja w IGV
</div>

In [ ]:
import matplotlib.pyplot  as plt
idx = (tsv_read.start > 10002294) & (tsv_read.start < 10002623)
plt.plot(tsv_read.start[idx], tsv_read.coverage[idx] , linewidth=15.0)
plt.axis([None, None, 0, 40])
fig=plt.figure(figsize=(150, 15))


In [ ]:
import igv

b = igv.Browser({"genome": "hg19"})
b.show()
b.search('20:10002294-10002623')
b.load_track(
    {
        "type": "alignment",
        "name": "mother BAM",
        "url": "../2-germline-vd/bams/mother.bam",
        "indexURL": "../2-germline-vd/bams/mother.bai",        
        "format": "bam",
        "indexed": True
    })


# Adnotacje

In [ ]:
from biothings_client import get_client
import pandas as pd
pd.set_option('display.max_columns', 2000)
mv = get_client('variant')
mv.getvariants(["chr7:g.140453134T>C","chr7:g.140453134T>C"], as_dataframe=1)


In [ ]:
df[1:2]

In [ ]:
list(mv.query('dbnsfp.genename:CDK2', fetch_all=True))[0]
#http://docs.myvariant.info/en/latest/doc/data.html#available-fields

In [ ]:
#mv.getvariants('chr17:g.7578532A>G', fields = ['clinvar','gnomad_af'] , as_dataframe=1)
gg = mv.getvariants('chr7:g.140453134T>C', fields = "all" , as_dataframe=1)
gg.shape
gg

#https://cdn.rawgit.com/biothings/myvariant.info/master/docs/ipynb/myvariant_clinvar_demo.html

In [ ]:
! gunzip ${WORKSPACE}/sandbox/motherHC.vcf
df = mv.getvariants(mv.get_hgvs_from_vcf(workspace + "/sandbox/motherHC.vcf") \
                    ,fields="all", as_dataframe=1, fetch_all=True)


In [ ]:
df[1:10]

In [ ]:
pd.options.display.max_seq_items = 2000
df.columns

In [ ]:
df["gnomad_genome.af.af_popmax"].plot(figsize=(20,10))

In [ ]:
df.groupby(['dbsnp.gene.name']).count()

# Poprawa genotypowania

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://us.v-cdn.net/5019796/uploads/FileUpload/40/a04b64d7884a07d1b562ead4002c60.jpg")

In [ ]:
! gatk CalculateGenotypePosteriors \
    -V ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -ped ${WORKSPACE}/trio.ped \
    --skip-population-priors \
    -O ${WORKSPACE}/sandbox/trioCGP.vcf

In [ ]:
! gatk CalculateGenotypePosteriors \
    -V ${WORKSPACE}/sandbox/trioGGVCF.vcf \
    -ped ${WORKSPACE}/trio.ped \
    --supporting-callsets ${WORKSPACE}/resources/af-only-gnomad.chr20subset.b37.vcf.gz \
    -O ${WORKSPACE}/sandbox/trioCGP_gnomad.vcf

In [ ]:
! bgzip /home/${USER}/2-germline-vd/sandbox/trioGGVCF.vcf
! tabix /home/${USER}/2-germline-vd/sandbox/trioGGVCF.vcf.gz
! bgzip /home/${USER}/2-germline-vd/sandbox/trioCGP.vcf
! tabix /home/${USER}/2-germline-vd/sandbox/trioCGP.vcf.gz
! bgzip /home/${USER}/2-germline-vd/sandbox/trioCGP_gnomad.vcf
! tabix /home/${USER}/2-germline-vd/sandbox/trioCGP_gnomad.vcf.gz


In [ ]:
import igv

b = igv.Browser({"genome": "hg19"})
b.show()

b.search('20:10002294-10002623')

b.load_track(
    {
        "type": "variant",
        "name": "combined gvcf",
        "url": "../edugen_genomics/sandbox/trioGGVCF.vcf.gz",
        "indexURL":  "../edugen_genomics/sandbox/trioGGVCF.vcf.gz.tbi",  
        "format": "vcf",
        "indexed": True
    })


b.load_track(
    {
        "type": "variant",
        "name": "combined gvcf",
        "url": "../edugen_genomics/sandbox/trioCGP.vcf.gz",
        "indexURL":  "../edugen_genomics/sandbox/trioCGP.vcf.gz.tbi",  
        "format": "vcf",        
        "indexed": True
    })


b.load_track(
    {
        "type": "variant",
        "name": "combined gvcf",
        "url": "../edugen_genomics/sandbox/trioCGP_gnomad.vcf.gz",
        "indexURL":  "../edugen_genomics/sandbox/trioCGP_gnomad.vcf.gz.tbi",  
        "format": "vcf",        
        "indexed": True
    })



# Ocena jakości wariantów przy użyciu CollectVariantCallingMetrics
 
CollectVariantCallingMetrics wylicza zbiorecze metryki na poziomie probki i genotypu. 

In [ ]:
! gatk CollectVariantCallingMetrics \
    -I  ${WORKSPACE}/sandbox/trioGGVCF.vcf.gz \
    --DBSNP ${WORKSPACE}/resources/dbsnp.vcf \
    -O ${WORKSPACE}/sandbox/trioGGVCF_metrics